## Graph Knowledge y RAG

### Ejemplo de nodos y relaciones

Nodo 1: Jugador - Alex Gómez  
Nodo 2: Jugador - Luis Fernández  
Nodo 3: Equipo - FC Rápidos  
Nodo 4: Evento - Gol en el minuto 45  

Relación 1: Alex Gómez --[pertenece a]--> FC Rápidos  
Relación 2: Luis Fernández --[pertenece a]--> FC Rápidos  
Relación 3: Alex Gómez --[anotó]--> Gol en el minuto 45  




### Comparación: Grafos de conocimiento vs. Bases de datos vectoriales

| Característica              | Grafos de conocimiento    | Bases de datos vectoriales       |
|-----------------------------|-----------------------------|--------------------------------|
| **Representación de datos** | Entidades y relaciones      | Vectores de alta dimensión     |
| **Mecanismos de recuperación** | Traversal de relaciones    | Similitud vectorial            |
| **Interpretabilidad**       | Alta                        | Baja                           |
| **Razonamiento**            | Complejo y explícito        | Limitado a similitud           |


### Implementación con llama-index

In [14]:
from langchain.llms import OpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
import os
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv())
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
%pip install json-repair

In [ ]:
llm = OpenAI(temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
%pip install neo4j

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
database = "neo4j"

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-graph-stores-neo4j
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-azure-openai

In [23]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore

In [80]:
documents = SimpleDirectoryReader("../data/games",
).load_data()

In [87]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

In [90]:
query_engine = index.as_query_engine(
    include_text=False, response_mode="tree_summarize"
)
response = query_engine.query("¿De qué equipo es Luis Fernández?")

In [91]:
response.response

'Luis Fernández es del equipo Fc rápidos.'